In [14]:
# Import packages
import pandas as pd 
import numpy as np 
import geopandas as gpd

from tqdm import tqdm
import requests
import io
import json
import plotly.express as px


In [5]:
districts = gpd.read_file("../data/external/WijkBuurtkaart_2020_v1/wijk_2020_v1.shp")
utrecht_dist = districts[['WK_NAAM','GM_NAAM','BEV_DICHTH','AANT_INW', 'AANT_MAN', 'AANT_VROUW','AANTAL_HH','OPP_TOT', 'OPP_LAND', 'OPP_WATER','geometry']][districts["GM_NAAM"]=="Utrecht"]
utrecht_dist

,WK_NAAM,GM_NAAM,BEV_DICHTH,AANT_INW,AANT_MAN,AANT_VROUW,AANTAL_HH,OPP_TOT,OPP_LAND,OPP_WATER,geometry
839,Wijk 01 West,Utrecht,3614,29270,14210,15055,15350,898,810,88,"POLYGON ((133755.081 456695.605, 133754.085 45..."
840,Wijk 02 Noordwest,Utrecht,10294,45255,22060,23195,24655,453,440,14,"POLYGON ((133828.685 457994.883, 133766.874 45..."
841,Wijk 03 Overvecht,Utrecht,4193,34295,17200,17095,16670,848,818,30,"POLYGON ((134335.839 458999.112, 134250.809 45..."
842,Wijk 04 Noordoost,Utrecht,7966,39680,18780,20895,21440,507,498,9,"POLYGON ((136313.674 457683.622, 136347.644 45..."
843,Wijk 05 Oost,Utrecht,2987,32080,14905,17170,20820,1104,1074,30,"POLYGON ((137357.132 455270.002, 137352.610 45..."
844,Wijk 06 Binnenstad,Utrecht,7371,19165,9815,9345,12755,271,260,11,"POLYGON ((135973.382 456367.876, 136051.013 45..."
845,Wijk 07 Zuid,Utrecht,6077,27895,13685,14210,14890,473,459,14,"POLYGON ((136374.068 453213.248, 136472.663 45..."
846,Wijk 08 Zuidwest,Utrecht,7635,38620,19430,19190,19985,555,506,49,"POLYGON ((134487.691 453674.259, 134356.758 45..."
847,Wijk 09 Leidsche Rijn,Utrecht,3944,41290,20685,20605,16355,1126,1047,79,"POLYGON ((131472.956 455779.536, 131472.911 45..."
848,Wijk 10 Vleuten-De Meern,Utrecht,1435,49795,24465,25330,18540,3685,3471,214,"POLYGON ((128020.999 456069.697, 128017.276 45..."


In [6]:
utrecht_dist["measurement_station"] = utrecht_dist.representative_point()

In [10]:
df = pd.read_csv("../data/external/resource_2020_06_15_2020_06_22.csv")
geo_df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['lon'],df['lat'], crs="EPSG:4326"))
geo_df = geo_df.to_crs("EPSG:28992")

geo_df = gpd.sjoin(geo_df, utrecht_dist, how="inner", op='within')

geo_df

,sensor,air_quality_observed_id,lon,lat,recording_time,trip_sequence,humidity,pm2_5,pressure,temperature,...,GM_NAAM,BEV_DICHTH,AANT_INW,AANT_MAN,AANT_VROUW,AANTAL_HH,OPP_TOT,OPP_LAND,OPP_WATER,measurement_station
870,0256bbbcd69440f79ee1a6bcb061d213,34786224,5.182125,52.088528,2020-06-15 18:29:52,0,54,2,1036,24.7,...,Utrecht,2987,32080,14905,17170,20820,1104,1074,30,POINT (138258.202 454388.051)
871,0256bbbcd69440f79ee1a6bcb061d213,34786225,5.181259,52.088402,2020-06-15 18:30:04,0,54,2,1036,24.7,...,Utrecht,2987,32080,14905,17170,20820,1104,1074,30,POINT (138258.202 454388.051)
872,0256bbbcd69440f79ee1a6bcb061d213,34786226,5.180094,52.088261,2020-06-15 18:30:16,0,54,2,1036,24.8,...,Utrecht,2987,32080,14905,17170,20820,1104,1074,30,POINT (138258.202 454388.051)
873,0256bbbcd69440f79ee1a6bcb061d213,34786227,5.178892,52.088078,2020-06-15 18:30:28,0,54,2,1037,24.8,...,Utrecht,2987,32080,14905,17170,20820,1104,1074,30,POINT (138258.202 454388.051)
874,0256bbbcd69440f79ee1a6bcb061d213,34786228,5.177822,52.087891,2020-06-15 18:30:40,0,54,2,1037,24.8,...,Utrecht,2987,32080,14905,17170,20820,1104,1074,30,POINT (138258.202 454388.051)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153662,e1ff5e6bfd3be8138b1d6513d636f01e,35396873,5.108903,52.095333,2020-06-21 16:01:19,22,54,4,1052,26.7,...,Utrecht,10294,45255,22060,23195,24655,453,440,14,POINT (134646.528 458056.888)
153663,e1ff5e6bfd3be8138b1d6513d636f01e,35396874,5.109118,52.095066,2020-06-21 16:01:31,22,54,3,1052,26.7,...,Utrecht,10294,45255,22060,23195,24655,453,440,14,POINT (134646.528 458056.888)
153665,e1ff5e6bfd3be8138b1d6513d636f01e,35396876,5.107979,52.094696,2020-06-21 16:01:55,22,54,2,1052,26.7,...,Utrecht,10294,45255,22060,23195,24655,453,440,14,POINT (134646.528 458056.888)
153666,e1ff5e6bfd3be8138b1d6513d636f01e,35396877,5.107298,52.094353,2020-06-21 16:02:07,22,54,3,1052,26.7,...,Utrecht,10294,45255,22060,23195,24655,453,440,14,POINT (134646.528 458056.888)


In [11]:
geo_df['recording_time'] = pd.to_datetime(df['recording_time'], format="%Y-%m-%d %H:%M:%S")
geo_df['date'] = geo_df['recording_time'].dt.date
geo_df['hour'] = geo_df['recording_time'].dt.hour

In [18]:
export = geo_df[["WK_NAAM", "date", "hour", "pm2_5"]].groupby(["WK_NAAM", "date", "hour"]).median().reset_index()
export["datetime"] = pd.to_datetime(export.date) + export.hour.astype('timedelta64[h]')
export

,WK_NAAM,date,hour,pm2_5,datetime
0,Wijk 01 West,2020-06-15,13,4.0,2020-06-15 13:00:00
1,Wijk 01 West,2020-06-15,15,4.0,2020-06-15 15:00:00
2,Wijk 01 West,2020-06-15,16,3.0,2020-06-15 16:00:00
3,Wijk 01 West,2020-06-15,17,3.0,2020-06-15 17:00:00
4,Wijk 01 West,2020-06-15,18,3.0,2020-06-15 18:00:00
...,...,...,...,...,...
631,Wijk 10 Vleuten-De Meern,2020-06-21,17,3.0,2020-06-21 17:00:00
632,Wijk 10 Vleuten-De Meern,2020-06-21,18,2.0,2020-06-21 18:00:00
633,Wijk 10 Vleuten-De Meern,2020-06-21,19,2.0,2020-06-21 19:00:00
634,Wijk 10 Vleuten-De Meern,2020-06-21,20,2.0,2020-06-21 20:00:00


In [16]:
df = export[["WK_NAAM", "date", "pm2_5"]].groupby(["WK_NAAM"]).count().reset_index().sort_values(by=['pm2_5'], ascending=False)

fig = px.bar(df, x="WK_NAAM", y="pm2_5", title="Total activity recorded per district")
fig.show()

In [17]:
df = export[["WK_NAAM", "date", "pm2_5"]].groupby(["WK_NAAM", "date"]).count().reset_index()

fig = px.bar(df, x='date', y='pm2_5', color="WK_NAAM")
fig.show()

In [19]:
fig = px.bar(export.groupby(["datetime"]).count().reset_index(), x='datetime', y='pm2_5', title='Measurements per hour', )
fig.show()

In [21]:
export = export.merge(utrecht_dist[["WK_NAAM", "geometry", "measurement_station"]], on="WK_NAAM")

export['lon'] = export.measurement_station.apply(lambda p: p.x)
export['lat'] = export.measurement_station.apply(lambda p: p.y)

export.to_csv("../data/interim/export_districts_2020_06_15_2020_06_22_hourly.csv",index=False)

In [22]:
export = geo_df[["WK_NAAM", "date", "hour", "pm2_5"]].groupby(["WK_NAAM", "date"]).median().reset_index()
export = export.merge(utrecht_dist[["WK_NAAM", "geometry", "measurement_station"]], on="WK_NAAM")

export['lon'] = export.measurement_station.apply(lambda p: p.x)
export['lat'] = export.measurement_station.apply(lambda p: p.y)

export.to_csv("../data/interim/export_districts_2020_06_15_2020_06_22_daily.csv",index=False)